In [8]:
# libraries
import pandas as pd
import numpy as np
from plotnine import *
import itertools
from scipy.stats import linregress

# data
file_path = "C:/Users/HelenSepman/OneDrive - Kruvelab/Documents/GitHub/ML_calibration_graph_linearity/0_data/data_ready_addfeatures_231128.csv"
# file_path = "C:/Users/yvkr1259/Documents/data_ready_addfeatures_231122.csv"

df_calibrations = pd.read_csv(file_path)
df_selected = df_calibrations[df_calibrations['lab']=='L1']
df_selected = df_selected[df_selected['compound']=='Tetraethylammonium']

### linear range iteratively

In [ ]:
def calculate_regression(points):
    x, y = zip(*points)
    slope, intercept, _, _, _ = linregress(x, y)
    y_pred = [slope * xi + intercept for xi in x]
    residuals = np.array(y) - np.array(y_pred)
    relative_residuals = np.abs(residuals) / np.abs(np.array(y))
    return relative_residuals.sum()

best_subset = None
best_residuals = float('inf')
data = df_selected[['c_real_M', 'peak_area']]

data.rename(columns={'c_real_M':'x'}, inplace=True)
data.rename(columns={'peak_area':'y'}, inplace=True)

# Iterate through all combinations of 3 to 6 points
for subset_size in range(3, 6):
    for subset in itertools.combinations(data.iterrows(), subset_size):
        subset_points = [(row[1]['x'], row[1]['y']) for _, row in subset]
        residuals_sum = calculate_regression(subset_points)
        if residuals_sum < best_residuals:
            best_residuals = residuals_sum
            best_subset = subset_points

# Output the best subset
print("Best Subset:", best_subset)
print("Best Relative Residuals:", best_residuals)


In [21]:
for subset_size in range(3, 6):
    for subset in itertools.combinations(data.iterrows(), subset_size):
        subset_points = [(row['x'], row['y']) for _, row in subset]
        residuals_sum = calculate_regression(subset_points)
        if residuals_sum < best_residuals:
            best_residuals = residuals_sum
            best_subset = subset_points

# Output the best subset
print("Best Subset:", best_subset)
print("Best Relative Residuals:", best_residuals)



Best Subset: [(1.13e-06, 1339187188.0), (3.59e-07, 226837232.7), (2.41e-07, 74981332.67)]
Best Relative Residuals: 0.14483422856421815


In [24]:
df_best = pd.DataFrame(best_subset)
slope = x
intercept = y
df_best['subset'] = i